In [ ]:
# code is inspired from : https://www.geeksforgeeks.org/stacking-in-machine-learning-2/

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [3]:
path = "/content/healthcare-dataset-stroke-data.csv"

df = pd.read_csv(path)

df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
X = df.copy()

X = X.drop(columns=['id','work_type','Residence_type','ever_married','gender'])

X.head()
print(X.shape)

(5110, 7)


In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(X['smoking_status'])
X['smoking_status'] = encoder.transform(X['smoking_status'])

X.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,smoking_status,stroke
0,67.0,0,1,228.69,36.6,1,1
1,61.0,0,0,202.21,NaN,2,1
2,80.0,0,1,105.92,32.5,2,1
3,49.0,0,0,171.23,34.4,3,1
4,79.0,1,0,174.12,24.0,2,1


In [6]:
X.dropna(inplace=True)

X.isnull().sum()

age                  0
hypertension         0
heart_disease        0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [7]:
y = X['stroke']
X = X.drop(columns='stroke')



In [8]:
from sklearn.preprocessing import StandardScaler

# normalize data to optimize model fit
data_scaler = StandardScaler()

X = data_scaler.fit_transform(X)

X

array([[ 1.07013796, -0.31806673,  4.38196829,  2.77769839,  0.98134488,
        -0.35178804],
       [ 1.64656262, -0.31806673,  4.38196829,  0.0138418 ,  0.45926914,
         0.58523176],
       [ 0.27201152, -0.31806673, -0.22820795,  1.48413156,  0.70120668,
         1.52225157],
       ...,
       [-0.34875349, -0.31806673, -0.22820795, -0.50236926,  0.21733161,
         0.58523176],
       [ 0.36069224, -0.31806673, -0.22820795,  1.37291993, -0.41934612,
        -0.35178804],
       [ 0.05030973, -0.31806673, -0.22820795, -0.45081569, -0.34294479,
        -1.28880785]])

In [9]:
X = X
y = y.values

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [11]:
KNC = KNeighborsClassifier()   # initialising KNeighbors Classifier
NB = GaussianNB()  

In [12]:
model_kNeighborsClassifier = KNC.fit(x_train, y_train)   # fitting Training Set
pred_knc = model_kNeighborsClassifier.predict(x_test)   # Predicting on test dataset

In [13]:
acc_knc = accuracy_score(y_test, pred_knc)  # evaluating accuracy score
print('accuracy score of KNeighbors Classifier is:', acc_knc * 100)

print(classification_report(y_test, pred_knc))

accuracy score of KNeighbors Classifier is: 95.43973941368078
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1176
           1       0.00      0.00      0.00        52

    accuracy                           0.95      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.92      0.95      0.94      1228



In [14]:
model_NaiveBayes = NB.fit(x_train, y_train)
pred_nb = model_NaiveBayes.predict(x_test)

In [15]:
acc_nb = accuracy_score(y_test, pred_nb)
print('Accuracy of Naive Bayes Classifier:', acc_nb * 100)

print(classification_report(y_test, pred_nb))

Accuracy of Naive Bayes Classifier: 88.59934853420195
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      1176
           1       0.15      0.37      0.21        52

    accuracy                           0.89      1228
   macro avg       0.56      0.64      0.58      1228
weighted avg       0.94      0.89      0.91      1228



In [16]:
KNC2 = KNeighborsClassifier(n_neighbors=4) 
KNC2.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [17]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100).fit(x_train, y_train)

In [18]:
pred_rf = RF.predict(x_test)

acc_rf = accuracy_score(y_test, pred_rf)
print('Accuracy of Random Forest Classifier:', acc_rf * 100)

print(classification_report(y_test, pred_rf))

Accuracy of Random Forest Classifier: 95.52117263843648
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1176
           1       0.29      0.04      0.07        52

    accuracy                           0.96      1228
   macro avg       0.62      0.52      0.52      1228
weighted avg       0.93      0.96      0.94      1228



In [19]:
from sklearn.svm import SVC

SV = SVC(probability=True)

SV.fit(x_train, y_train)

pred_sv = SV.predict(x_test)

acc_sv = accuracy_score(y_test, pred_sv)
print('Accuracy of Support Vector Classifier:', acc_sv * 100)

print(classification_report(y_test, pred_sv))

Accuracy of Support Vector Classifier: 95.76547231270358
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1176
           1       0.00      0.00      0.00        52

    accuracy                           0.96      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.92      0.96      0.94      1228



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
import sklearn
from sklearn.linear_model import LinearRegression
lr = LogisticRegression(max_iter=300)  # defining meta-classifier
clf_stack = StackingClassifier(classifiers =[RF, NB], meta_classifier = lr, use_probas = True, use_features_in_secondary = True)

In [21]:

model_stack = clf_stack.fit(x_train, y_train)   # training of stacked model
pred_stack = model_stack.predict(x_test)

In [22]:

acc_stack = accuracy_score(y_test, pred_stack)  # evaluating accuracy
print('accuracy score of Stacked model:', acc_stack * 100)

accuracy score of Stacked model: 95.43973941368078


In [23]:


print(classification_report(y_test, pred_stack))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1176
           1       0.38      0.12      0.18        52

    accuracy                           0.95      1228
   macro avg       0.67      0.55      0.58      1228
weighted avg       0.94      0.95      0.94      1228

